# OCR automation tutorial 

Tutorial presents how to use Optical Character Recognition(OCR) model to automate text annotations. We will use bounding box annotations annotated in the [SuperAnnotate](https://app.superannotate.com/) platform to do text prediction on them and merge predictions to the annotations. Here we use [EasyOCR](https://github.com/JaidedAI/EasyOCR) model to get predictions.

In [1]:
import os.path
import tempfile
import cv2
import superannotate as sa
import easyocr as eo

# Initialize SuperAnnotate SDK

Below you need initialize your SuperAnnotate SDK. You need to calll `init` function and pass configuration JSON file as an argument. The file should contain token of your team. To avoid initialization everytime you can set `confg.json` file in `~/.superannotate/` folder and it will automatically read from that path. SuperAnnotate SDK detailed information can be found [here](https://superannotate.readthedocs.io/en/stable/index.html).

In [2]:
# Initialize SuperAnnotate SDK
# path_to_config = '<your_config_file_path>'
# sa.init(path_to_config)

path_to_config = '~/.superannotate/webinar_config.json'
sa.init(path_to_config)

# Define variables

Define project name and folder name that you want to use for OCR predictions.

In [9]:
# Define your variables
# project_name = '<your_project_name>'
# folder_name = '<your_folder_name>'

project_name = 'ocr_example_project'
folder_name = 'test-dataset'

In [10]:
project_path = project_name
if folder_name:
    project_path = f'{project_name}/{folder_name}'

In [11]:
project_path

'ocr_example_project/imgs'

# List images

SuperAnnotate SDK allows us to search images with the statuses that we are interested. In this case I listed images that have annotation status `QualityCheck`.

In [12]:
# Get all images from the project
imgs_list = sa.search_images(project_path, annotation_status='InProgress')

# Load EasyOCR for predictions

[EasyOCR](https://github.com/JaidedAI/EasyOCR) is an open source OCR model for text recognition in the image. It supports 80+ languages and you also can combine multiple language predictions. However, in our case its only english text in the annotation project. 

In [13]:
# Get EasyOCR model for English text
reader = eo.Reader(['en'])

# Download, predict and upload annotations

In [14]:
sa_annotations = sa.get_annotations(project_path, imgs_list)
# Loop over all images and set text for each bounding box
for annotation in sa_annotations:
    # Create a temporary Directory to download images there
    with tempfile.TemporaryDirectory() as tmp_dirname:
        # Download image to the following directory
        sa.download_image(project_path, annotation['metadata']['name'], tmp_dirname)
        # Open image with OpenCV lib
        img = cv2.imread(os.path.join(tmp_dirname, annotation['metadata']['name']))
        # Loop over all instances
        for instance in annotation['instances']:
            if instance['type'] != 'bbox':
                continue
            # Get bounding box sizes from annotations
            bbox = instance['points']
            # Crop image with bounding box
            cropped_img = img[int(bbox['y1']):int(bbox['y2']) + 1, int(bbox['x1']):int(bbox['x2']) + 1]
            # Run EasyOCR model on cropped image
            results = reader.readtext(cropped_img)
            # Get all words from the prediction that have more than 0.1 confidence
            words = [prediction[1] for prediction in results if prediction[2] > 0.1]
            # Make a line from the predicted words
            line = ' '.join(words) + '\n'
            # Set the line as pointLabel of the bounding box
            instance['pointLabels']['0'] = line
        # Upload annotation back to the project
        sa.upload_image_annotations(project_path, annotation['metadata']['name'], annotation) 


SA-PYTHON-SDK - INFO - Getting 6 annotations from ocr_example_project/imgs.


Processing: 100%|██████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.78it/s]


SA-PYTHON-SDK - INFO - Downloaded image 1.jpg to /tmp/tmp2q82jyyw 
SA-PYTHON-SDK - INFO - Uploading annotations for image 1.jpg in project ocr_example_project.
SA-PYTHON-SDK - INFO - Downloaded image 2.png to /tmp/tmp561vxabe 
SA-PYTHON-SDK - INFO - Uploading annotations for image 2.png in project ocr_example_project.
SA-PYTHON-SDK - INFO - Downloaded image 3.jpg to /tmp/tmpbfmd_sve 
SA-PYTHON-SDK - INFO - Uploading annotations for image 3.jpg in project ocr_example_project.
SA-PYTHON-SDK - INFO - Downloaded image 4.jpg to /tmp/tmp9qz6shg9 
SA-PYTHON-SDK - INFO - Uploading annotations for image 4.jpg in project ocr_example_project.
SA-PYTHON-SDK - INFO - Downloaded image 5.jpg to /tmp/tmpxs_t5utx 
SA-PYTHON-SDK - INFO - Uploading annotations for image 5.jpg in project ocr_example_project.
SA-PYTHON-SDK - INFO - Downloaded image 6.jpg to /tmp/tmpfk_5ud7g 
SA-PYTHON-SDK - INFO - Uploading annotations for image 6.jpg in project ocr_example_project.
